In [6]:
from unidecode import unidecode

l = """Máy bay nào đến thành phố Huế lúc 13:30HR ?
Máy bay nào bay từ Đà Nẵng đến TP. Hồ Chí Minh mất 1 giờ ?
Hãy cho biết mã hiệu các máy bay hạ cánh ở Huế ?
Máy bay nào xuất phát từ Tp.Hồ Chí Minh, lúc mấy giờ ?
Máy bay nào bay từ TP.Hồ Chí Minh đến Hà Nội ?
Máy bay VN4 có xuất phát từ Đà Nẵng không ?
Thời gian máy bay VJ5 bay từ TP. Hà Nội đến Khánh Hòa mất mấy giờ ?
Có máy bay nào xuất phát từ Hải Phòng không ?
Máy bay của hãng hàng không VietJet Air bay đến những thành phố nào ?
Có máy bay nào bay từ Hải Phòng đến Khánh Hòa không ?"""

l = l.split('\n')

In [7]:
from utils import *
l = [normalize_sentence(s) for s in l]
l

['máy bay nào đến thành phố huế lúc 13:30hr ?',
 'máy bay nào bay từ đà nẵng đến thành phố  hồ chí minh mất 1 giờ ?',
 'hãy cho biết mã hiệu các máy bay hạ cánh ở huế ?',
 'máy bay nào xuất phát từ thành phố hồ chí minh, lúc mấy giờ ?',
 'máy bay nào bay từ thành phố hồ chí minh đến hà nội ?',
 'máy bay vn4 có xuất phát từ đà nẵng không ?',
 'thời gian máy bay vj5 bay từ thành phố  hà nội đến khánh hòa mất mấy giờ ?',
 'có máy bay nào xuất phát từ hải phòng không ?',
 'máy bay của hãng hàng không vietjet air bay đến những thành phố nào ?',
 'có máy bay nào bay từ hải phòng đến khánh hòa không ?']

In [9]:
idx_POS_regex_list = get_idx_POS_regex_list(l[0])
idx_POS_regex_list

[(0, 7, 'FLIGHT-N', 'may bay', 'máy bay'),
 (8, 11, 'QDET', 'nao', 'nào'),
 (12, 15, ['FLIGHT_END-V', 'TO-P'], 'den', 'đến'),
 (16, 25, 'CITY-N', 'thanh pho', 'thành phố'),
 (26, 29, 'CITY-NAME', 'hue', 'huế'),
 (30, 33, 'AT-P', 'luc', 'lúc'),
 (34, 41, 'FLIGHT_TIME-N', '13:30hr', '13:30hr'),
 (42, 43, 'PUNC', '?', '?')]

In [10]:
relations = parse_dependency(idx_POS_regex_list)
draw_relation(relations)

'qdet: máy bay -> nào\nnsubj: đến -> máy bay\nroot: ROOT -> đến\nnmod: huế -> thành phố\nobl: đến -> huế\ncase: 13:30hr -> lúc\nobl: đến -> 13:30hr\npunc: đến -> ?\n'

In [11]:
relations

[('qdet',
  (0, 7, 'FLIGHT-N', 'may bay', 'máy bay'),
  (8, 11, 'QDET', 'nao', 'nào')),
 ('nsubj',
  (12, 15, ['FLIGHT_END-V', 'TO-P'], 'den', 'đến'),
  (0, 7, 'FLIGHT-N', 'may bay', 'máy bay')),
 ('root', 'ROOT', (12, 15, ['FLIGHT_END-V', 'TO-P'], 'den', 'đến')),
 ('nmod',
  (26, 29, 'CITY-NAME', 'hue', 'huế'),
  (16, 25, 'CITY-N', 'thanh pho', 'thành phố')),
 ('obl',
  (12, 15, ['FLIGHT_END-V', 'TO-P'], 'den', 'đến'),
  (26, 29, 'CITY-NAME', 'hue', 'huế')),
 ('case',
  (34, 41, 'FLIGHT_TIME-N', '13:30hr', '13:30hr'),
  (30, 33, 'AT-P', 'luc', 'lúc')),
 ('obl',
  (12, 15, ['FLIGHT_END-V', 'TO-P'], 'den', 'đến'),
  (34, 41, 'FLIGHT_TIME-N', '13:30hr', '13:30hr')),
 ('punc',
  (12, 15, ['FLIGHT_END-V', 'TO-P'], 'den', 'đến'),
  (42, 43, 'PUNC', '?', '?'))]

In [7]:
def get_grammatical_relations(relations):
    gramatical_relations = []

    def add_gramatical_relations(var, gramma, word):
        gramatical_relations.append((var, gramma, word))

    root = [x for x in relations if x[0] == 'root'][0]
    add_gramatical_relations('s1', 'PRED', root[2])
    nsubj = [x for x in relations if x[0] == 'nsubj'][0]
    add_gramatical_relations('s1', 'LSUBJ', nsubj[2])
    dobj = [x for x in relations if x[0] == 'dobj']
    if len(dobj) != 0:
        add_gramatical_relations('s1', 'LOBJ', dobj[0][2])

    obls = [x for x in relations if x[0] == 'obl']
    for obl in obls:
        if obl[1][2] in [['FLIGHT_END-V', 'TO-P'], 'FLIGHT_END-V']:
            if obl[2][2] in ['CITY-NAME']:
                add_gramatical_relations('s1', 'TO', obl[2])
            elif obl[2][2] in ['FLIGHT_TIME-N']:
                add_gramatical_relations('s1', 'AT-TIME', obl[2])
            
    
    cases = [x for x in relations if x[0] == 'case']
    for case in cases:
        if case[1][2] in ['CITY-NAME', 'CITY-N']:
            if case[2][2] in ['FROM-P']:
                add_gramatical_relations('s1', 'FROM', case[1])
            if case[2][2] in [['FLIGHT_END-V', 'TO-P']]:
                add_gramatical_relations('s1', 'TO', case[1])
        if case[1][2] in ['HOUR-N']:
            if case[2][2] in ['AT-P']:
                add_gramatical_relations('s1', 'AT-TIME', case[1])
    
    advmods = [x for x in relations if x[0] == 'advmod']
    for advmod in advmods:
        if advmod[2][3] == 'mat':
            unit = [x for x in relations if x[0] == 'dobj' and x[1][3] == 'mat'][0][2][3]
            num = [x for x in relations if x[0] == 'nummod' and x[1][3] == unit][0][2][3]
            unit = 'HR' if unit=='gio' else unit
            add_gramatical_relations('s1', 'TAKE', f'{num}:00{unit}')
    

    acls =   [x for x in relations if x[0] == 'acl']      
    for acl in acls:
        add_gramatical_relations('s2', 'LSUBJ', acl[1])
        add_gramatical_relations('s2', 'PRED', acl[2])
        stack_add = []
        stack_delete = []
        for ele in gramatical_relations:
            if ele[1] not in ['PRED', 'LSUBJ', 'LOBJ']:
                stack_add.append(('s2', ele[1], ele[2]))
                stack_delete.append(ele)
        [gramatical_relations.pop(gramatical_relations.index(ele)) for ele in stack_delete]
        gramatical_relations += stack_add
    
    return gramatical_relations

def draw_grammatical_relations(gramatical_relations):
    res = ""
    for i in gramatical_relations:
        res += f'({i[0]} {i[1]} "{i[2][4]}")\n'
    return res

idx_POS_regex_list = get_idx_POS_regex_list(l[3])
relations = parse_dependency(idx_POS_regex_list)
gramatical_relations = get_grammatical_relations(relations)
draw_grammatical_relations(gramatical_relations)
        

'(s1 PRED "xuất phát")\n(s1 LSUBJ "máy bay")\n(s1 FROM "hồ chí minh")\n(s1 AT-TIME "giờ")\n'

In [21]:
def get_logical_form(gramatical_relations, relations):
    source = []
    dest = []
    run_time = []
    arrive = []
    depart = []
        
    lsubj = [x for x in gramatical_relations if x[1] == 'LSUBJ'][0]
    try: 
        lsubj = [x for x in gramatical_relations if x[1] == 'LSUBJ'][1]
    except:
        pass
    if lsubj[2][2] == 'FLIGHT-NAME':
        flight = ['FLIGHT', lsubj[2][3]]
    else:
        flight = ['FLIGHT', 'f1']
    
    pred = [x for x in gramatical_relations if x[1] == 'PRED'][0]
    if pred[2][2] == 'LAST-P':
        run_time = ['RUN-TIME', 'r1', 't3']
    


    if 'AT-TIME' in [x[1] for x in gramatical_relations]:
        name_time = [x[2][4] for x in gramatical_relations if x[1] == 'AT-TIME'][0]
        pred = [x for x in gramatical_relations if x[1] == 'PRED'][0]
        if pred[2][2] in [['FLIGHT_END-V', 'TO-P']]:
            arrive = ['ARRIVE', 'a1', f'(NAME t2 "{name_time}")']
        elif pred[2][2] == 'FLIGHT_BEGIN-V':
            depart = ['DEPART', 'd1', f'(NAME t1 "{name_time}")']
    
    if 'TO' in [x[1] for x in gramatical_relations]:
        name_to = [x[2][4] for x in gramatical_relations if x[1] == 'TO'][0]
        dest = ['DEST', flight[1], f'(NAME c2 "{name_to}")']
    
    if 'FROM' in [x[1] for x in gramatical_relations]:
        name_source = [x[2][4] for x in gramatical_relations if x[1] == 'FROM'][0]
        source = ['SOURCE', flight[1], f'(NAME c1 "{name_source}")']
    
    if 'TAKE' in [x[1] for x in gramatical_relations]:
        name_hour = [x[2] for x in gramatical_relations if x[1] == 'TAKE'][0]
        run_time = ['RUN-TIME', 'r1', f'(NAME t3 "{name_hour}")']

    #what element the question ask?
    vars_question = []
    try:
        qdets = [x for x in relations if x[0] == 'qdet']
        for qdet in qdets:
            if qdet[1][2] == 'FLIGHT-N':
                vars_question.append(flight[1])
            elif qdet[1][2] == 'HOUR-N':
                l = [depart, arrive, run_time]
                vars_question.append(f't{1+l.index([x for x in l if x!= []][0])}')
            elif qdet[1][2] == 'CITY-N':
                eles = [x for x in gramatical_relations if x[1] == 'FROM']+[x for x in gramatical_relations if x[1] == 'TO']
                for ele in eles:
                    if ele[2][2] != 'CITY-NAME' and ele[1] == 'TO':
                        vars_question.append('c2')
                    elif ele[2][2] != 'CITY-NAME' and ele[1] == 'FROM':
                        vars_question.append('c1')
    except:
        vars_question.append(flight[1])

    if len(qdets) == 0: # yes no question
        flight[1] = 'f1'
        if source != []:
            source[1] = 'f1'
        if dest != []:
            dest[1] = 'f1'
        vars_question.append(flight[1])
    
    return [vars_question, flight, source, dest, arrive, depart, run_time]

def draw_logical_form(args):
    def list2string(l):
        if len(l)==0:
            return ''
        s = ''
        for e in l:
            s += e + ' '
        s = s.strip()
        return f'({s})'
    
    def list2string_vars_question(l):
        s = ''
        for e in l:
            s += e + ', '
        return s[:-2]
    vars_question, flight, source, dest, arrive, depart, run_time = args
    return f'WH {list2string_vars_question(vars_question)}: (&{list2string(flight)}{list2string(source)}{list2string(dest)}){list2string(arrive)}{list2string(depart)}{list2string(run_time)}\n'
        
idx_POS_regex_list = get_idx_POS_regex_list(l[4])
relations = parse_dependency(idx_POS_regex_list)
gramatical_relations = get_grammatical_relations(relations)
logical_form_args = get_logical_form(gramatical_relations)
draw_logical_form(logical_form_args)


'WH f1: (&(FLIGHT f1)(SOURCE f1 (NAME c1 "hồ chí minh"))(DEST f1 (NAME c2 "hà nội")))\n'

In [31]:
def get_procedural_sematic(logical_form_args):
    vars_question, flight, source, dest, arrive, depart, run_time = logical_form_args

    Flight = []
    Atime = []
    Dtime = []
    Runtime = []

    pattern = re.compile('(vn|vj)\d')
    Flight = ['FLIGHT', flight[1].upper() if pattern.match(flight[1]) else '?f1']

    #handle Dtime
    city1 = '?c1'
    time1 = '?t1'
    if len(source) != 0:
        name_city = source[2].split('"')[1]
        cities_vietnam = ['hồ chí minh', 'huế', 'hà nội', 'đà nẵng', 'hải phòng', 'khánh hòa']
        city_codes = ['HCMC', 'HUE', 'HN', 'ĐN', 'HP', 'KH']
        if name_city in cities_vietnam:
            city1 = city_codes[cities_vietnam.index(name_city)]
    if len(depart) != 0:
        name_hour = depart[2].split('"')[1].upper()
        pattern = re.compile("\d+:\d+HR")
        if pattern.match(name_hour):
            time1 = name_hour
    Dtime = ['DTIME', Flight[1], city1, time1]    
    
    #handle Atime
    city2 = '?c2'
    time2 = '?t2'
    if len(dest) != 0:
        name_city = dest[2].split('"')[1]
        cities_vietnam = ['hồ chí minh', 'huế', 'hà nội', 'đà nẵng', 'hải phòng', 'khánh hòa']
        city_codes = ['HCMC', 'HUE', 'HN', 'ĐN', 'HP', 'KH']
        if name_city in cities_vietnam:
            city2 = city_codes[cities_vietnam.index(name_city)]
    if len(arrive) != 0:
        name_hour = arrive[2].split('"')[1].upper()
        pattern = re.compile("\d+:\d+HR")
        if pattern.match(name_hour):
            time2 = name_hour
    Atime = ['ATIME', Flight[1], city2, time2]

    time3 = '?t3'
    if len(run_time) != 0:
        name_hour = run_time[2] if run_time[2]=='t3' else run_time[2].split('"')[1].upper()
        pattern = re.compile("\d+:\d+HR")
        if pattern.match(name_hour):
            time3 = name_hour
    Runtime = ['RUN-TIME', Flight[1], city1, city2, time3]

    vars_question = [f'?{x}' for x in vars_question]

    
    return [vars_question, Flight, Atime, Dtime, Runtime]

def draw_procedural_sematic(procedural_sematic_res):
    def list2string(l, f=0):
        if all(x[0]=='?' for x in l[1:]) and f == 0:
            return ''
        if f == 2:
            if not all(x[0]!='?' for x in l[2:-1]):
                return ''
        s = ''
        for e in l:
            s += e + ' '
        s = s.strip()
        return f'({s})'
    
    def list2string_vars_question(l):
        s = ''
        for e in l:
            s += e + ', '
        return s[:-2]
    vars_question, Flight, Atime, Dtime, Runtime = procedural_sematic_res
    res = f'(PRINT-ALL {list2string_vars_question(vars_question)} {list2string(Flight, 1)} {list2string(Dtime)} {list2string(Atime)} {list2string(Runtime,2)})\n'
    res = f"{re.sub(' +', ' ',res).strip()}\n"
    return res

idx_POS_regex_list = get_idx_POS_regex_list(l[9])
relations = parse_dependency(idx_POS_regex_list)
gramatical_relations = get_grammatical_relations(relations)
logical_form_res = get_logical_form(gramatical_relations)      
procedural_sematic_res = get_procedural_sematic(logical_form_res)
draw_procedural_sematic(procedural_sematic_res)

'(PRINT-ALL ?f1 (FLIGHT ?f1) (DTIME ?f1 HP ?t1) (ATIME ?f1 KH ?t2) (RUN-TIME ?f1 HP KH ?t3))\n'

In [11]:
procedural_sematic_res

[['?f1'],
 ['FLIGHT', '?f1'],
 ['ATIME', '?f1', 'KH', '?t2'],
 ['DTIME', '?f1', 'HP', '?t1'],
 ['RUN-TIME', '?f1', 'HP', 'KH', '?t3']]

In [12]:
db = open('input/database.txt', 'r',  encoding="utf8").read()
db = [a.strip().replace('(','').replace(')','').split(' ') for a in db.split('\n')]

In [25]:
def retrieve_database(procedural_sematic_res):
    global db
    vars_question, Flight, Atime, Dtime, Runtime = procedural_sematic_res

    def no_consts(cond, is_runtime = 0):
        a = len ([x for x in cond if x[0]=='?'])
        return 4 - a if is_runtime else 3 - a
    
    def match(cond_db, cond_query):
        if len(cond_db) != len(cond_query):
            return False
        for idx in range(len(cond_db)):
            if cond_query[idx][0] !='?':
                if cond_query[idx] != cond_db[idx]:
                    return False
        return True
    
    num_of_const = [no_consts(Dtime), no_consts(Atime), no_consts(Runtime, 1)]

    idx_handle = num_of_const.index(max(num_of_const))
    matches=[]

    if idx_handle == 0: #handle DTime
        matches = [x for x in db if match(x, Dtime)]
    if idx_handle == 1: #handle ATime
        matches = [x for x in db if match(x, Atime)]
    if idx_handle == 2: #handle RumTime
        matches = [x for x in db if match(x, Runtime)]

    #handle vars_question
    answers_dict = {}
    for var in vars_question:
        answer = set()
        if var == '?f1':
            for a in [x[1] for x in matches]:
                answer.add(a)
        if var == '?t1':
            for a in [x[3] for x in matches if x[0]=='DTIME']:
                answer.add(a)
        if var == '?t2':
            for a in [x[3] for x in matches if x[0]=='ATIME']:
                answer.add(a)
        if var == '?t3':
            for a in [x[4] for x in matches if x[0]=='RUN-TIME']:
                answer.add(a)
        answers_dict.update({var:list(answer)})
    
    return answers_dict

def draw_answer(answers_dict):
    def list2string_vars_question(l):
        s = ''
        for e in l:
            s += e + ', '
        return s[:-2]
    res = ''
    for key in answers_dict.keys():
        keys1 = ['?f1', '?t1', '?t2', '?t3']
        keys2 = ['Flights', 'Depart Time', 'Arrive Time', 'Run Time']
        res += f'{keys2[keys1.index(key)]}: {list2string_vars_question(answers_dict[key])}\n'
    return res




In [1]:
from utils import *
l = """Máy bay nào đến thành phố Huế lúc 13:30HR ?
Máy bay nào bay từ Đà Nẵng đến TP. Hồ Chí Minh mất 1 giờ ?
Hãy cho biết mã hiệu các máy bay hạ cánh ở Huế ?
Máy bay nào xuất phát từ Tp.Hồ Chí Minh, lúc mấy giờ ?
Máy bay nào bay từ TP.Hồ Chí Minh đến Hà Nội ?
Máy bay VN4 có xuất phát từ Đà Nẵng không ?
Thời gian máy bay VJ5 bay từ TP. Hà Nội đến Khánh Hòa mất mấy giờ ?
Có máy bay nào xuất phát từ Hải Phòng không ?
Máy bay của hãng hàng không VietJet Air bay đến những thành phố nào ?
Có máy bay nào bay từ Hải Phòng đến Khánh Hòa không ?"""

l = l.split('\n')
l = [normalize_sentence(s) for s in l]

db = open('input/database.txt', 'r',  encoding="utf8").read()
db = [a.strip().replace('(','').replace(')','').split(' ') for a in db.split('\n')]

In [3]:
idx_POS_regex_list = get_idx_POS_regex_list(l[0])
relations = parse_dependency(idx_POS_regex_list)
gramatical_relations = get_grammatical_relations(relations)
logical_form_res = get_logical_form(gramatical_relations, relations)      
procedural_sematic_res = get_procedural_sematic(logical_form_res)
answers_dict = retrieve_database(procedural_sematic_res, db) 

print(draw_relation(relations))
print(draw_grammatical_relations(gramatical_relations))
print(draw_logical_form(logical_form_res))
print(draw_procedural_sematic(procedural_sematic_res))
print(draw_answer(answers_dict))

qdet: máy bay -> nào
nsubj: đến -> máy bay
root: ROOT -> đến
nmod: huế -> thành phố
obl: đến -> huế
case: 13:30hr -> lúc
obl: đến -> 13:30hr
punc: đến -> ?

(s1 PRED "đến")
(s1 LSUBJ "máy bay")
(s1 TO "huế")
(s1 AT-TIME "13:30hr")

WH f1: (&(FLIGHT f1)(DEST f1 (NAME c2 "huế")))(ARRIVE a1 (NAME t2 "13:30hr"))

(PRINT-ALL ?f1 (FLIGHT ?f1) (ATIME ?f1 HUE 13:30HR) )

Flights: VJ1



In [27]:
"output_query_{:4d}".format(1)

'output_query_   1'